<a href="https://colab.research.google.com/github/Nicolas-Ryd/Google-Colab-identificaci-n-de-imagenes-Gato-y-Perro/blob/main/Gato_Perro_Versi%C3%B3n_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1. Fases de inferencia
 En esta primera fase, prepararemos el entorno para el funcionamiento del resto del código y nuestra inteligencia artificial detectorra de perros y gatos, definiendo ambas categorias para su posterior clasificación.

 En primer lugar, se habilitamos el modo legacy Keras, necesario para utilizar modelos entrenados con versiones anteriores de google colab.  

 Después, el programa carga el modelo ya entrenado. Este modelo contiene toda la información aprendida durante el proceso de entrenamiento, es decir, los patrones visuales que permiten diferenciar un gato de un perro. También, se cargan las etiquetas de las clases, lo que permite saber qué nombre corresponde a cada salida del modelo.

In [ ]:
import os
# 1. Activamos el modo legacy (Keras 2)
os.environ['TF_USE_LEGACY_KERAS'] = '1'

from tensorflow import keras

#Carga el modelo
mi_modelo = keras.models.load_model("/content/drive/MyDrive/keras_model.h5", compile=False)

# Carga las etiquetas de las clases
nombre_clases = open("/content/drive/MyDrive/labels.txt", "r").readlines()

print(nombre_clases)


['0 Perro\n', '1 Gato\n']


#Fase 2: Adquirir Datos Nuevos
Ahora lo que introduciremos, será la siguiente linea de código para poder subir a google drive las imagenes de prueba que deseamos que el modelo clasifique.

In [ ]:
from PIL import Image
imagen= Image.open("/content/drive/MyDrive/Ejercicios perras y gatas/cats_dogs_light/test/cat.9828.jpg").convert("RGB")

#Fase 3: Pre-procesamiento
En esta fase, la imagen debe transformarse para ser idéntica a las de entrenamiento por lo que debemos pasar las imagenes de clasificación al mismo tamaño y caracteristicas que las imagenes de enrenamiento, de lo contrario el modelo no funcionará correctamente. Es decir, se procesa para adaptarse a los requisitos del modelo: se redimensiona a un tamaño específico, se ajusta su composición y se normalizan los valores de sus píxeles.

In [ ]:
from PIL import Image, ImageOps
import numpy as np
size = (224, 224)
imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)
imagen_array = np.asarray(imagen)
normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1

#Fase 4: Predicción
En esta fase, crearemos un conjunto de imagenes para que sea analizado por la IA organizando las imagenes en un lote de datos, ya que el modelo espera recibir las imágenes en colecciones

In [ ]:
lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
lote_imagenes[0] = normalizada_imagen_array
resultados= mi_modelo.predict(lote_imagenes)

1/1 [==============================] - 3s 3s/step


# Fase 5: Post-procesamiento
Esta quinta fase se ejecuta el proceso de predicción, en el que el modelo analiza la imagen y genera una probabilidad para cada clase. Finalmente, se identifica cuál es la clase con mayor probabilidad y se muestra su nombre y el nivel de probabildad.

In [ ]:
indice = np.argmax(resultados[0])
etiqueta = nombre_clases[indice]
print("La imagen es de clase: ", etiqueta)
probabilidad = resultados[0][indice]
print("Con una probabilidad de: ", probabilidad)

La imagen es de clase:  1 Gato

Con una probabilidad de:  0.9999993


#Paso Intermedio
###Definir predecir_imagen
En esta sección de nuestro programa, se define una función encargada de automatizar todo este proceso de inferencia para cualquier imagen. Esta función permite reutilizar el código y predicción sin repetirlo por cada imagen. Además, limpia el formato de la etiqueta para entregar un resultado claro y comprensible al usuario.



In [ ]:
def predecir_imagen(ruta_imagen):
    imagen = Image.open(ruta_imagen).convert("RGB")
    size = (224, 224)
    imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)


    imagen_array = np.asarray(imagen)
    normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1
    lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    lote_imagenes[0] = normalizada_imagen_array

    resultados = mi_modelo.predict(lote_imagenes, verbose=0)
    indice = np.argmax(resultados[0])
    etiqueta = nombre_clases[indice]
    etiqueta_limpia = etiqueta.split(' ', 1)[1].strip().lower()

    probabilidad = resultados[0][indice]

    return etiqueta_limpia, probabilidad

print("La función 'predecir_imagen' ha sido refactorizada.")

La función 'predecir_imagen' ha sido refactorizada.


#Fase 6: Preparación para lectura de carpetas
Posteriormente, el programa recorre la carpeta de “test” con cientos de imágenes. Para cada archivo, deduce la etiqueta esperada basándose en su nombre, prepara la imagen y solicita una predicción al modelo. Si la predicción coincide con la etiqueta real, se considera un acierto; si no, se registra como un error. Al mismo tiempo, se acumulan las probabilidades de acierto para calcular una media final.

In [ ]:
lista_archivos = os.listdir("/content/drive/MyDrive/Ejercicios perras y gatas/cats_dogs_light/test")
total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
# predicciones incorrectas contendrá una lista con los nombres de las imágenes mal clasificadas
predicciones_incorrectas = []

for nombre_archivo in lista_archivos:
    if "cat" in nombre_archivo:
        etiqueta_esperada = "gato"
    elif "dog" in nombre_archivo:
        etiqueta_esperada = "perro"
    else:
        continue  # Saltar archivos que no sean de gatos o perros

    total_predicciones += 1

    ruta_imagen = os.path.join("/content/drive/MyDrive/Ejercicios perras y gatas/cats_dogs_light/test", nombre_archivo)

    # Aquí obtendriamos la predicción del modelo llamando a una función que
    # implemente las fases de inferencia
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha,
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)


# Fase 7: Cálculo de métricas y generación del informe
Finalmente, cuando todas las imágenes han sido evaluadas, el programa calcula la precisión final con todos los aciertos y errores que ha tenido el modelo. También calcula la probabilidad media de los aciertos y genera un informe completo que incluye cuántas imágenes fueron clasificadas correctamente y cuáles fueron identificadas de manera incorrecta, indicando la predicción errónea y su probabilidad.

In [ ]:
precision = aciertos / total_predicciones if total_predicciones > 0 else 0
media_probabilidad_aciertos /= aciertos if aciertos > 0 else 1
informe = f"""
Informe de Evaluación del Modelo

Total de predicciones: {total_predicciones}
Aciertos: {aciertos}
Precisión: {precision:.2%}
Probabilidad media de aciertos: {media_probabilidad_aciertos:.2%}

Predicciones incorrectas:
"""

for error in predicciones_incorrectas:
    informe += f"Archivo: {error['archivo']}, Prediccion: {error['prediccion']}, Probabilidad: {error['probabilidad']:.2%}\n"

print(informe)


Informe de Evaluación del Modelo

Total de predicciones: 400
Aciertos: 390
Precisión: 97.50%
Probabilidad media de aciertos: 99.30%

Predicciones incorrectas:
Archivo: dog.9808.jpg, Prediccion: gato, Probabilidad: 84.00%
Archivo: dog.9868.jpg, Prediccion: gato, Probabilidad: 93.13%
Archivo: dog.9911.jpg, Prediccion: gato, Probabilidad: 99.96%
Archivo: dog.9935.jpg, Prediccion: gato, Probabilidad: 99.96%
Archivo: cat.9836.jpg, Prediccion: perro, Probabilidad: 98.81%
Archivo: cat.9874.jpg, Prediccion: perro, Probabilidad: 84.85%
Archivo: cat.9882.jpg, Prediccion: perro, Probabilidad: 98.40%
Archivo: cat.9954.jpg, Prediccion: perro, Probabilidad: 79.82%
Archivo: cat.9960.jpg, Prediccion: perro, Probabilidad: 91.54%
Archivo: cat.9947.jpg, Prediccion: perro, Probabilidad: 92.40%

